In [ ]:
%%capture
!pip install datasets transformers tqdm
!pip install accelerate -U
!pip install huggingface_hub

In [ ]:
%%capture
!pip install flash_attn==2.3.4 Pillow==10.3.0 requests==2.31.0 torch==2.3.0 torchvision==0.18.0

In [ ]:
import pandas as pd
import numpy as np
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from datasets import Dataset
import ast
from huggingface_hub import login


In [ ]:
# Enter your Hugging Face API token
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/tar_projekt/csvs/dataset_validation_allLevels.csv')
test_df = test_df.loc[test_df['level'] > 1]

texts = test_df['text']
labels = test_df['change']
labels = [ast.literal_eval(string) for string in labels]

test_data = []

for text, label in zip(texts, labels):
    paragraphs = text.split(' ||| ')
    for (p1, p2), y in zip(zip(paragraphs, paragraphs[1: ]), label):
        p1 = p1.strip()
        p2 = p2.strip()

        test_data.append({'text': p1 + '\n' + p2, 'label': y})
test_data = Dataset.from_pandas(pd.DataFrame(test_data))

In [ ]:
paragraphs_df = pd.read_csv('/content/drive/MyDrive/tar_projekt/labels_per_paragraph.csv')
paragraphs_df

,text-id,text,label,version
0,medium-17,"Due to the overreach on the initial layoffs, c...",1,3
1,medium-17,I used to be in Pharmacy. Then I did a researc...,0,3
2,medium-17,So if a medical center has any that are partia...,1,3
3,medium-17,Open vials are supposed to be discarded after ...,1,3
4,medium-17,It costs about 15-35 cents to make a mL of ins...,1,3
...,...,...,...,...
161,hard-862,"Well, I still don’t have an answer. I have opt...",0,6
162,hard-862,"Same here, others too. I don’t consider email ...",1,6
163,hard-862,After months of email going to spam I finally ...,1,6
164,hard-862,Receiving the same e-mails from Cedars trying ...,0,6


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/6.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
 # Define your question prompt
question = paragraphs_df['text'][0]

# Format the input as a T5 sequence-to-sequence task
input_text = "Return 0 if both paragraphs are written by the same author, otherwise return 1: " + question

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
outputs = model.generate(input_ids=inputs["input_ids"],
                          attention_mask=inputs["attention_mask"],
                          max_length=100, num_beams=3)

# Decode the generated response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

0


In [ ]:
responses = []

for pair in paragraphs_df['text']:
  # Define your question prompt
  question = pair

  # Format the input as a T5 sequence-to-sequence task
  input_text = "Return 0 if both paragraphs are written by the same author, otherwise return 1: " + question

  # Tokenize the input text
  inputs = tokenizer(input_text, return_tensors="pt")

  # Generate a response
  outputs = model.generate(input_ids=inputs["input_ids"],
                          attention_mask=inputs["attention_mask"],
                          max_length=100, num_beams=1)

  # Decode the generated response
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  responses.append(response)

In [ ]:
responses = [int(x) for x in responses]

In [ ]:
paragraphs_df['flan-t5-xxl_prediction'] = responses

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
# Calculate F1 score
real_labels = paragraphs_df['label']
predicted_labels = responses

f1 = f1_score(real_labels, predicted_labels)

# Calculate confusion matrix
cm = confusion_matrix(real_labels, predicted_labels)

print("F1 Score:", f1)
print("Confusion Matrix:")
print(cm)

F1 Score: 0.6556016597510372
Confusion Matrix:
[[ 4 64]
 [19 79]]


In [ ]:
paragraphs_df.to_csv('flan_t5_predictions.csv')